# pg_recall

a time machine for your PostgreSQL data

In [1]:
\connect host=localhost user=manuel database=postgres nopassword

ok

## what it's for

keeps track of data changes to your tables

- look at past states of your tables

- list changes for individual keys

- allows you to implement tools to automate that
 - undo/redo buttons
 - Revision history page
 - a backend for customer service where they can see what the user did
 - look for data/users that behave oddly

for...
- user data 
- logging metrics

... data that doesn't change too often

(*often* being very malleable here)

## design goals

CRUD transparency

simplicity

flexibility

little overhead

## design choices

storage is (relatively) cheap
- logs rows, not individual fields
- detects unchanged rows
- each table has a retention interval

separate log tables

timestamps identify revisions

In [ ]:
BEGIN;
SELECT now();

In [ ]:
SELECT now();

In [ ]:
ROLLBACK;

`tstzrange` as revision identifier

no constraints in the log table

## Restrictions

it protects user data, not schema changes

manual cleanup

storage overhead

depends on `btree_gist`

## Functions

In [ ]:
SELECT recall.enable('myTable', '3 months');
SELECT recall.enable('someSchema.otherTable', '1 year', 'targetSchema');

In [ ]:
SELECT recall.disable('tableName');

In [ ]:
SELECT recall.at('myTable', now() - interval '2 months')
SELECT * FROM myTable_past;

In [ ]:
SELECT recall.cleanup('tableName');
SELECT recall.cleanup_all();

## Demo

![table schema](/files/blog_tables.png)

In [ ]:
BEGIN; -- run all this in a transaction (for predictable timing)

In [ ]:
CREATE TABLE IF NOT EXISTS account (
  uid SERIAL PRIMARY KEY,
  created_at TIMESTAMPTZ NOT NULL DEFAULT now(),

  name VARCHAR(200) NOT NULL,
  login VARCHAR(100) NOT NULL,
  password VARCHAR(200) NOT NULL,
  email VARCHAR(200) NOT NULL
);
CREATE UNIQUE INDEX IF NOT EXISTS idx_account_login ON account(lower(login));

CREATE TABLE IF NOT EXISTS account_settings (
  uid INTEGER NOT NULL,
  key VARCHAR(100) NOT NULL,
  value TEXT NOT NULL,

  PRIMARY KEY (uid, key),
  FOREIGN KEY (uid) REFERENCES account(uid)
);

CREATE TABLE IF NOT EXISTS blog_entry (
  entry_id SERIAL PRIMARY KEY,
  creator INTEGER NOT NULL,
  created_at TIMESTAMPTZ NOT NULL DEFAULT now(),

  title VARCHAR(200) NOT NULL,
  content TEXT NOT NULL,

  FOREIGN KEY (creator) REFERENCES account(uid)
);

Installation
------------

run `make install` in the source directory

In [ ]:
CREATE EXTENSION IF NOT EXISTS btree_gist;
CREATE EXTENSION IF NOT EXISTS recall WITH VERSION '0.9.5';

SELECT recall.enable('account_settings', '3 months');
SELECT recall.enable('blog_entry', '6 months');

In [ ]:
-- cheating a little
CREATE OR REPLACE FUNCTION pretendToWait(t INTERVAL) RETURNS void AS $$
  UPDATE recall.account_settings_log SET _log_time = tstzrange(LOWER(_log_time)-t, UPPER(_log_time)-t);
  UPDATE recall.blog_entry_log SET _log_time = tstzrange(LOWER(_log_time)-t, UPPER(_log_time)-t);
$$ LANGUAGE sql;

In [ ]:
\dt

In [ ]:
\dt recall

### Some data...

In [ ]:
INSERT INTO account (uid, name, login, password, email)
VALUES (12, 'John Doe', 'jdoe', 'very secure password', 'jdoe@example.com')
RETURNING uid;

In [ ]:
INSERT INTO account_settings (uid, key, value) VALUES
(12, 'get_newsletter', true),
(12, 'enable_spellcheck', false);

In [ ]:
INSERT INTO blog_entry (entry_id, creator, title, content) VALUES
(123, 12, 'Welcome to my new bog', 'This is sooooo super exciting!'),
(124, 12, 'House warming party', 'I want to invite you all to my house warming party next tuesday at 123 Some Place')
RETURNING entry_id;

### Changes

In [ ]:
-- fix a typo
SELECT pretendToWait('5 minutes');
UPDATE blog_entry SET title = 'Welcome to my new blog' WHERE entry_id = 123;

In [ ]:
-- enable spell check to prevent typos in the future
SELECT pretendToWait('5 minutes');
UPDATE account_settings SET value = true WHERE uid = 12 AND key = 'enable_spellcheck';

In [ ]:
-- remove the second blog entry
SELECT pretendToWait('5 minutes');
DELETE FROM blog_entry WHERE entry_id = 124;

### Let's have a look

In [ ]:
SELECT * FROM account;

In [ ]:
SELECT * FROM account_settings;

In [ ]:
SELECT * FROM blog_entry;

### Going back in time

In [ ]:
SELECT recall.at('blog_entry', now() - interval '10 minutes');
SELECT * FROM blog_entry_past;

### Behind the scenes

In [ ]:
SELECT uid, key, value, _log_time::text FROM recall.account_settings_log;

In [ ]:
SELECT entry_id, creator, created_at, title, content, _log_time::text FROM recall.blog_entry_log;

In [ ]:
SELECT * FROM recall._config;

In [ ]:
ROLLBACK;

## PostgreSQL features

extension support

range types

- no-overlap

- non-empty

GiST¹ indexes

¹ Generalized Search Tree

the `btree_gist` extension contains GiST index implementations for built in types

table inheritance

`pl/pgsql`

## Future

- automatic partitioning
- ports?
- ...

## Similar projects

### PostgreSQL

- [TimeTravel for PostgreSQL][6] (GNU GPLv3)
- [A PL/pgSQL Trigger Procedure For Auditing][7] in the PostgreSQL docs

### Others

- Temporal queries in SQL:2011
- [Oracle FlashBack][8]
- [CouchDB's Document Revisions][9]
- [EclipseLink JPA History][10]

[6]: http://www.databtech.com/eng/index_timetravel.htm
[7]: http://www.postgresql.org/docs/current/static/plpgsql-trigger.html#PLPGSQL-TRIGGER-AUDIT-EXAMPLE
[8]: https://docs.oracle.com/cd/B28359_01/appdev.111/b28424/adfns_flashback.htm
[9]: http://docs.couchdb.org/en/1.6.1/intro/api.html#revisions
[10]: https://wiki.eclipse.org/EclipseLink/Examples/JPA/History

## Questions?

feel free to talk to me afterwards, tweet me at [@mreithub][1] or send an email to [manuel@reithuber.net][2]

[1]: https://twitter.com/mreithub
[2]: mailto:manuel@reithuber.net?subject=pg_recall